Logistic map:

$f(x) = 4x(1-x)$

In [1]:
f(x) = 4x*(1-x)

f (generic function with 1 method)

In [2]:
using IntervalConstraintProgramming, ValidatedNumerics

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.4/IntervalConstraintProgramming.ji for module IntervalConstraintProgramming.


In [3]:
C = @constraint 4x*(1-x) ∈ [0.3, 0.4]

Separator with variables x

In [4]:
X = IntervalBox(0..1)

[0, 1]

In [5]:
paving = setinverse(C, X)

Paving:
- tolerance ϵ = 0.01
- inner approx. of length 4
- boundary approx. of length 4

In [6]:
paving.inner

4-element Array{ValidatedNumerics.IntervalBox{1,Float64},1}:
 [0.901015, 0.916761] 
 [0.889014, 0.901016] 
 [0.0989849, 0.110986]
 [0.0832394, 0.098985]

In [7]:
paving.boundary

4-element Array{ValidatedNumerics.IntervalBox{1,Float64},1}:
 [0.91676, 0.918335]   
 [0.886832, 0.889015]  
 [0.110985, 0.113168]  
 [0.0816653, 0.0832395]

Inverse functions:


In [8]:
g1(x) = (1/2) * (1 + √(1-x))
g2(x) = (1/2) * (1 - √(1-x))

g2 (generic function with 1 method)

In [12]:
Y = IntervalBox(hull(g2(Interval(0.3)), g2(Interval(0.4))))

[0.0816699, 0.112702]

In [13]:
setdiff(Y, paving)

0-element Array{ValidatedNumerics.IntervalBox{1,Float64},1}

Second iterate:

In [14]:
C2 = @constraint 4*(4x*(1-x))*(1-(4x*(1-x))) ∈ [0.3, 0.4]

Separator with variables x

In [15]:
paving = setinverse(C2, X)

Paving:
- tolerance ϵ = 0.01
- inner approx. of length 18
- boundary approx. of length 14

In [16]:
paving.inner

18-element Array{ValidatedNumerics.IntervalBox{1,Float64},1}:
 [0.974218, 0.978592]  
 [0.971488, 0.974219]  
 [0.657081, 0.660659]  
 [0.666639, 0.666798]  
 [0.660658, 0.663409]  
 [0.652046, 0.657082]  
 [0.646114, 0.647525]  
 [0.648568, 0.652047]  
 [0.645094, 0.646115]  
 [0.342918, 0.347954]  
 [0.353885, 0.354906]  
 [0.347953, 0.351432]  
 [0.352475, 0.353886]  
 [0.339341, 0.342919]  
 [0.336591, 0.339342]  
 [0.333202, 0.333361]  
 [0.0257812, 0.0285113]
 [0.021408, 0.0257813] 

Inverses:

In [17]:
h1(x) = (1/2) * (1 - √(1 - √(1 - x)) / √2)

h1 (generic function with 1 method)

In [18]:
Y = IntervalBox(hull(h1(Interval(0.3)), h1(Interval(0.4))))

[0.332144, 0.357111]

In [19]:
setdiff(Y, paving)

0-element Array{ValidatedNumerics.IntervalBox{1,Float64},1}